In [1]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Constants:
    TAX_PATH = "./tax.docx"
    PINECONE_INDEX_NAME = "tax-index"
    MODEL_CHATGPT = "gpt-4o-mini"
    MODEL_EMBEDDING = "text-embedding-3-large"


In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서를 쪼갠다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 내부 문서 객체 하나가 가질 수 있는 토큰 수
    chunk_overlap=200 # 위/아래 문맥을 겹치게 하여 문서 객체 사이의 겹치는 토큰의 정도 (유사도를 위해)
)

loader = Docx2txtLoader(Constants.TAX_PATH)
document = loader.load_and_split(text_splitter=text_splitter)
len(document) # 187개

187

In [3]:
# 임베딩을 한다.
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model=Constants.MODEL_EMBEDDING)

In [ ]:
# import time
# index_name = "langchain-index" # 바꿔도 됨
#
# existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
#
# if index_name in existing_indexes:
#     pc.create_index(
#         name=index_name,
#         dimension=3072,
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1"),
#     )
#     while not pc.describe_index(index_name).status["ready"]:
#         time.sleep(1)
#
# index = pc.Index(index_name)

In [ ]:
import os
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pinecone_api_key

from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(
    documents=document,
    embedding=embedding,
    index_name=Constants.PINECONE_INDEX_NAME,
)

In [5]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

# 임베딩을 활용하여 알아서 유사도 검색 후 자료를 가져온다.
# 기본적으로 4개를 가져오는데 k 값을 바꾸면 된다.
# retrieved_docs = database.similarity_search(query, k=4)

# database 상관없이 가져오려면 아래처럼 하면 된다.
retriever = database.as_retriever(search_kwargs={'k': 4})
retrieved_docs = retriever.invoke(query)

retrieved_docs

[Document(id='85fa3cae-89dd-4150-a23d-2b9b490fc527', metadata={'source': './tax.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=Constants.MODEL_CHATGPT)

In [7]:
from langchain_core.prompts import PromptTemplate

template = """[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

rag_template = PromptTemplate(
    template=template,
    input_variables=["retrieved_docs", "query"])
rag_chain = rag_template | llm
ai_message = rag_chain.invoke({
    "retrieved_docs": retrieved_docs,
    "query": query
})

In [47]:
# prompt = f"""[Identity]
# - 당신은 최고의 한국 소득세 전문가 입니다.
# - [Context]를 참고해서 사용자의 질문에 답변해주세요.
#
# [Context]
# {retrieved_docs}
#
# Question: {query}
# """

# ai_message = llm.invoke(prompt)

In [8]:
ai_message.content

'연봉 5천만원인 직장인의 소득세는 다음과 같이 계산할 수 있습니다. 한국의 소득세는 누진세율 구조를 가지고 있으며, 기본적으로 과세표준에 따라 세율이 적용됩니다. \n\n2023년 기준 소득세 누진세율은 다음과 같습니다:\n\n1. 1,200만원 이하: 6%\n2. 1,200만원 초과 4,600만원 이하: 15%\n3. 4,600만원 초과 8,800만원 이하: 24%\n4. 8,800만원 초과 1억 5,000만원 이하: 35%\n5. 1억 5,000만원 초과: 38%\n\n연봉이 5천만원인 경우, 아래와 같이 과세표준을 나누어 세금을 계산합니다.\n\n1. 첫 번째 구간: 1,200만원 × 6% = 72만원\n2. 두 번째 구간: (4,600만원 - 1,200만원) × 15% = 5,100만원 × 15% = 765만원\n3. 세 번째 구간: (5,000만원 - 4,600만원) × 24% = 400만원 × 24% = 96만원\n\n전체 세액:\n- 72만원 + 765만원 + 96만원 = 933만원\n\n하지만 여기서 기본공제를 고려해야 합니다. 직장인의 근로소득공제는 총급여에 따라 다르며, 5천만원의 연봉에 대해 약 700만원 정도의 근로소득공제를 적용받을 수 있습니다. 즉, 최종 과세표준은 5,000만원 - 700만원(근로소득공제) = 4,300만원이 됩니다.\n\n따라서 최종적으로 4,300만원에 대한 세금을 계산합니다:\n- 1,200만원 × 6% = 72만원\n- (4,600만원 - 1,200만원) × 15% = 5,100만원 × 15% = 765만원\n- (4,300만원 - 4,600만원) × 24% = 0만원\n\n최종 세액 계산:\n- 72만원 + 765만원 (추가세 없음) = 837만원\n\n따라서 연봉 5천만원인 직장인의 대략적인 소득세는 약 837만원입니다. 단, 세액공제(예: 개인연금저축, 의료비, 교육비 등) 등이 추가로 적용될 수 있으며, 이로 인해 최종 납부세액은 달라질 수 있습니다. 정확한 세액은 개인의 상황에 따라 변동이 있을 수 